In [38]:
import pydal
from pydal import DAL, Field
import random

In [39]:
db = DAL('sqlite://storage.sqlite')

In [40]:
db.define_table('student',
                Field('name', 'string'),
                Field('email', 'string'),
                Field('n_quarters', 'integer'),
                Field('gpa', 'double'),
)

db.commit()

In [41]:
def random_string(n):
    return ''.join(random.choice('abcdefghijklmnopqrstuvwxyz') for _ in range(n))

In [42]:
for _ in range(10):
    db.student.insert(
        name=random_string(10),
        email=random_string(10) + '@example.com',
        n_quarters=random.randint(1, 10),
        gpa=random.uniform(0, 4),
    )
db.commit()

In [43]:
rows = db(db.student.gpa > 2).select(db.student.email, db.student.gpa, orderby=~db.student.gpa)

In [44]:
emails = [(row.email, row.gpa) for row in rows]

In [45]:
emails

[('nyrzzjdmlg@example.com', 4.0),
 ('oqkpvvkeqc@example.com', 4.0),
 ('wcnbzrxteb@example.com', 3.942348967360528),
 ('wwfcnxtuyc@example.com', 3.696785507783898),
 ('kpyovyeile@example.com', 3.483794537696732),
 ('lfgtuieolq@example.com', 2.964234735220358),
 ('dolfwrfxlj@example.com', 2.5427531040570734)]

In [46]:
# Update method 1.
student1 = 'oqkpvvkeqc@example.com'
student2 = 'nyrzzjdmlg@example.com'

s1 = db(db.student.email == student1).select().first()
s1.gpa = 4
s1.update_record()

<Row {'id': 9, 'name': 'yptzovbejf', 'email': 'oqkpvvkeqc@example.com', 'n_quarters': 1, 'gpa': 4}>

In [47]:
db(db.student.email == student2).update(gpa=4)

1

In [48]:
db(db.student.gpa < 2).delete()
db.commit()

## Representing relations

In [49]:
db.define_table('sighting',
    Field('date_of_sighting', 'date'),
    Field('location', 'string'),
    Field('species', 'string'),
    Field('count', 'integer'),
)
db.commit()

In [50]:
db.sighting.insert(
    date_of_sighting='2021-05-01',
    location='Seattle',
    species='Robin',
    count=1,
)
db.sighting.insert(
    date_of_sighting='2021-05-01',
    location='Seattle',
    species='Turkey',
    count=10,
)
db.commit()

## Birdwatching tables

In [58]:
db_birds = DAL('sqlite://birds.sqlite')

In [59]:
db_birds.define_table(
    'outing', # This table has an id as its key. 
    Field('date_of_outing', 'date'),
    Field('location', 'string'),
    Field('weather', 'string'),
    )

db_birds.define_table(
    'species',
    Field('english_name', 'string'),
    Field('latin_name', 'string'),
    Field('is_threatened', 'boolean'),
)

db_birds.define_table(
    'sighting',
    Field('outing_id', 'reference outing'), # Called a "foreign key"
    Field('species_id', 'reference species'),
    Field('count', 'integer'),
    )

db_birds.commit()

### Inserting a checklist

In [60]:
import datetime

In [61]:
date = datetime.date.today()
location = 'Santa Cruz'
weather = 'Sunny'
what = [
    ('Robin', 3),
    ('Turkey', 2),
    ('Sparrow', 1)
]

In [62]:
# First, insert the outing.
outing_id = db_birds.outing.insert(
    date_of_outing=date,
    location=location,
    weather=weather,
)

In [64]:
# Insert the sightings. 
for species_name, count in what:
    # Gets the species id, inserting the species if needed.
    species = db_birds(db_birds.species.english_name == species_name).select().first()
    if species is None:
        species_id = db_birds.species.insert(
            english_name=species_name,
            latin_name='Unknown',
            is_threatened=False,
        )
    else:
        species_id = species.id
    # Insert the sighting.
    db_birds.sighting.insert(
        outing_id=outing_id,
        species_id=species_id,
        count=count,
    )
db_birds.commit()

## Reading information across multiple tables

In [69]:
rows = db_birds(
    (db_birds.sighting.outing_id == db_birds.outing.id) & # join 1
    (db_birds.sighting.species_id == db_birds.species.id) # join 2
).select().as_list()


In [70]:
rows

[{'sighting': {'id': 1, 'outing_id': 1, 'species_id': 1, 'count': 3},
  'species': {'id': 1,
   'english_name': 'Robin',
   'latin_name': 'Unknown',
   'is_threatened': False},
  'outing': {'id': 1,
   'date_of_outing': datetime.date(2024, 10, 28),
   'location': 'Santa Cruz',
   'weather': 'Sunny'}},
 {'sighting': {'id': 2, 'outing_id': 1, 'species_id': 2, 'count': 2},
  'species': {'id': 2,
   'english_name': 'Turkey',
   'latin_name': 'Unknown',
   'is_threatened': False},
  'outing': {'id': 1,
   'date_of_outing': datetime.date(2024, 10, 28),
   'location': 'Santa Cruz',
   'weather': 'Sunny'}},
 {'sighting': {'id': 3, 'outing_id': 1, 'species_id': 3, 'count': 1},
  'species': {'id': 3,
   'english_name': 'Sparrow',
   'latin_name': 'Unknown',
   'is_threatened': False},
  'outing': {'id': 1,
   'date_of_outing': datetime.date(2024, 10, 28),
   'location': 'Santa Cruz',
   'weather': 'Sunny'}}]

## Registrar

In [56]:
dbr = DAL('sqlite://ucsc.sqlite')

In [57]:
dbr.define_table(
    'cat_course',
    Field('number', 'string'),
    Field('description', 'text'),
    Field('course_name', 'string'),   
)

dbr.define_table(
    'quarter',
    Field('season', 'string'),
    Field('year', 'integer'),
)

dbr.define_table(
    'instructor',
    Field('name', 'string'),
    Field('email', 'string'),
)

dbr.define_table(
    'offering',
    Field('course_id', 'reference cat_course'),
    Field('quarter_id', 'reference quarter'),
    Field('instructor_id', 'reference instructor'),
    Field('enrollment_limit', 'integer'),
)

dbr.define_table(
    'student',
    Field('name', 'string'),
    Field('email', 'string'),
)

dbr.define_table(
    'enrollment',
    Field('student_id', 'reference student'),
    Field('offering_id', 'reference offering'),
    Field('for_letter_grade', 'boolean'),
)

dbr.commit()